In [18]:
import numpy as np
import pandas as pd

## step 1: initialize rows and columns

In [5]:
# build scoring matrix of sequences
seq1 = "GCATGCT"
seq2 = "GATACCA"

n_rows = len(seq1)
n_columns = len(seq2)

scoring_matrix = np.zeros((n_rows + 1, n_columns + 1))
print("Scoring matrix:\n",scoring_matrix)

Scoring matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [13]:
# fill first row and columns
scoring_matrix[:,0] = np.linspace(0, -n_rows, n_rows + 1)
scoring_matrix[0,:] = np.linspace(0, -n_columns, n_columns + 1)
print("Scoring matrix:\n",scoring_matrix)

Scoring matrix:
 [[ 0. -1. -2. -3. -4. -5. -6. -7.]
 [-1.  0.  0.  0.  0.  0.  0.  0.]
 [-2.  0.  0.  0.  0.  0.  0.  0.]
 [-3.  0.  0.  0.  0.  0.  0.  0.]
 [-4.  0.  0.  0.  0.  0.  0.  0.]
 [-5.  0.  0.  0.  0.  0.  0.  0.]
 [-6.  0.  0.  0.  0.  0.  0.  0.]
 [-7.  0.  0.  0.  0.  0.  0.  0.]]


## step 2: fill the matrix with optimal score

In [16]:
# Useful constants
gap_penalty = -1
match_award = 1
mismatch= -1

# define a function returns the marks for base alignment
def matching(base1, base2):
    if base1 == base2:
        return match_award
    elif base1 == '-' or base2 == "-":
        return gap_penalty
    else:
        return mismatch

In [17]:
for i in range(1, n_rows + 1):
    for j in range(1, n_columns + 1):
        # calculate and compare all three cases:
        match = scoring_matrix[i - 1][j - 1] + matching(seq1[i - 1], seq2[j - 1])
        seq1_gap = scoring_matrix[i][j - 1] + gap_penalty
        seq2_gap = scoring_matrix[i - 1][j] + gap_penalty
        scoring_matrix[i,j] = max(match, seq1_gap, seq2_gap)
print("Scoring matrix:\n",scoring_matrix)


Scoring matrix:
 [[ 0. -1. -2. -3. -4. -5. -6. -7.]
 [-1.  1.  0. -1. -2. -3. -4. -5.]
 [-2.  0.  0. -1. -2. -1. -2. -3.]
 [-3. -1.  1.  0.  0. -1. -2. -1.]
 [-4. -2.  0.  2.  1.  0. -1. -2.]
 [-5. -3. -1.  1.  1.  0. -1. -2.]
 [-6. -4. -2.  0.  0.  2.  1.  0.]
 [-7. -5. -3. -1. -1.  1.  1.  0.]]


In [35]:
# print it with sequencing data
row_names = [0] + list(seq1)
col_names = [0] +list(seq2)
score_table = pd.DataFrame(scoring_matrix, columns=col_names, index=row_names)
score_table

,0,G,A,T,A,C,C,A
0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0
G,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0
C,-2.0,0.0,0.0,-1.0,-2.0,-1.0,-2.0,-3.0
A,-3.0,-1.0,1.0,0.0,0.0,-1.0,-2.0,-1.0
T,-4.0,-2.0,0.0,2.0,1.0,0.0,-1.0,-2.0
G,-5.0,-3.0,-1.0,1.0,1.0,0.0,-1.0,-2.0
C,-6.0,-4.0,-2.0,0.0,0.0,2.0,1.0,0.0
T,-7.0,-5.0,-3.0,-1.0,-1.0,1.0,1.0,0.0


## step 3: tracing back

In [36]:
# tracing back with two tracing strings
seq1_trac = ""
seq2_trac = ""

# from the bottom right corner
i = n_rows
j = n_columns
while i > 0 and j > 0: # end touching the top or the left edge
    score = scoring_matrix[i][j]
    diagonal = scoring_matrix[i-1][j-1]
    up = scoring_matrix[i-1][j]
    left = scoring_matrix[i][j-1]
        
    # Check what is previous score
    if score == diagonal + matching(seq1[i-1], seq2[j-1]):
        seq1_trac += seq1[i-1]
        seq2_trac += seq2[j-1]
        i -= 1
        j -= 1
    elif score == up + gap_penalty:
        seq1_trac += seq1[i-1]
        seq2_trac += '-'
        i -= 1
    elif score == left + gap_penalty:
        seq1_trac += '-'
        seq2_trac += seq2[j-1]
        j -= 1

    # Finish tracing up to the top left cell
while j > 0:
    seq1_trac += '-'
    seq2_trac += seq2[j-1]
    j -= 1
while i > 0:
    seq1_trac += seq1[i-1]
    seq2_trac += '-'
    i -= 1
    
# the records are reversed
align_seq1 = seq1_trac[::-1]
align_seq2 = seq2_trac[::-1]
    

print(align_seq1)
print(align_seq2)

GCAT-GCT
G-ATACCA
